In [5]:
import os
from time import time
from pathlib import Path
import pandas as pd 
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import Dataset
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [6]:
# Obtenemos la carpeta actual
current_dir = Path.cwd()
DATASETS_LOCATION = os.path.join(current_dir.parent.parent.parent, 'datasets')
MODELS_LOCATION = os.path.join(current_dir.parent.parent, 'models')

Cargamos el dataset:

In [7]:
df_disease_sp = pd.read_excel(os.path.join(DATASETS_LOCATION, 'disease_nlp_esp.xlsx'))

# Mostrar el DataFrame
df_disease_sp.head(3)

,Síntomas,Enfermedad
0,Me siento cansado todo el tiempo y he notado q...,Anemia
1,Últimamente me falta el aire incluso después d...,Anemia
2,"Durante las últimas semanas, me siento extrema...",Anemia


In [9]:
print("Dataset Disease Spanish antes de undersampling:")
print(df_disease_sp['Enfermedad'].value_counts())


Dataset Language distribution antes de undersampling:
Enfermedad
Anemia       100
Talasemia    100
Trombosis    100
Diabetes     100
Name: count, dtype: int64


'# Determinamos la cantidad mínima de ejemplos por clase\nmin_count = min(df_spanish_english[\'Language\'].value_counts())\n\n# Verificamos el equilibrio\nprint("\n Dataset Language distribution despues de undersampling:")\nprint(df_balanced[\'Language\'].value_counts())'

Observamos que ya las clases están bien distribuidas. Dividimos el dataset en conjunto de entrenamiento y prueba:

In [10]:
# Dividiamo il dataset bilanciato in train (80%) e test (20%)
train_df, test_df = train_test_split(df_disease_sp, test_size=0.2, random_state=42, stratify=df_disease_sp['Enfermedad'])

# Dividiamo ulteriormente il train in train (80%) e validation (20%)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Enfermedad'])

# Verifichiamo che i set siano equilibrati
print("\nConjunto de entrenamiento:")
print(train_df['Enfermedad'].value_counts())

print("\nConjunto de validación:")
print(val_df['Enfermedad'].value_counts())

print("\nConjunto de prueba:")
print(test_df['Enfermedad'].value_counts())



Conjunto de entrenamiento:
Enfermedad
Trombosis    64
Anemia       64
Talasemia    64
Diabetes     64
Name: count, dtype: int64

Conjunto de validación:
Enfermedad
Trombosis    16
Talasemia    16
Anemia       16
Diabetes     16
Name: count, dtype: int64

Conjunto de prueba:
Enfermedad
Talasemia    20
Diabetes     20
Anemia       20
Trombosis    20
Name: count, dtype: int64


In [11]:
# Crear un codificador de etiquetas para convertir las clases de texto en números
label_encoder = LabelEncoder()

# Convertir las clases de 'Enfermedad' a números
train_df['label'] = label_encoder.fit_transform(train_df['Enfermedad'])
val_df['label'] = label_encoder.transform(val_df['Enfermedad'])
test_df['label'] = label_encoder.transform(test_df['Enfermedad'])

train_df.head()

,Síntomas,Enfermedad,label
281,"He notado que mi pierna está hinchada, y las v...",Trombosis,3
22,"Estoy teniendo muchas infecciones, y mi piel s...",Anemia,0
146,"Últimamente me siento muy débil, y mi piel est...",Talasemia,2
70,He notado que el blanco de mis ojos tiene un t...,Anemia,0
376,"Tengo la vista borrosa, siempre tengo sed y pa...",Diabetes,1


## Entrenamiento

In [12]:
# Definimos la función de métricas
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average='macro'),
        "recall": recall_score(p.label_ids, preds, average='macro'),
        "f1": f1_score(p.label_ids, preds, average='macro'),
    }

In [15]:
# Definimos el modelo y el tokenizer
model_checkpoint = "bertin-project/bertin-roberta-base-spanish"  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maria\.cache\huggingface\hub\models--bertin-project--bertin-roberta-base-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Preprocesamos los datos
def preprocess_function(examples):
    return tokenizer(examples['Síntomas'], examples['Enfermedad'], padding=True, truncation=True, return_tensors="pt")

In [17]:
# Converti i DataFrame in dataset di Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(val_df)
# Tokenizamos el dataset
train_preprocessed_dataset = train_dataset.map(preprocess_function, batched=True)
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)
val_preprocessed_dataset = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,              # Tasa de aprendizaje: se ha probado 5e-6, 1e-5, 5e-5
    per_device_train_batch_size=32,   # Tamaño del batch para entrenamiento: se ha probado 8, 16, 32, 64
    per_device_eval_batch_size=32,    # Tamaño del batch para evaluación
    num_train_epochs=3,
    weight_decay=0.3,               # Decaimiento de peso: hemos probado 0.01, 0.1, 0.2 e 0.3 e 0.4 y este era el mejor
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_preprocessed_dataset,
    eval_dataset=val_preprocessed_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics   # Función para calcular las métricas
)

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\maria\AppData\Local\Temp\ipykernel_11364\2732148159.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training

In [19]:
train_preprocessed_dataset

Dataset({
    features: ['Síntomas', 'Enfermedad', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 256
})

In [20]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.196861982345581, 'eval_accuracy': 0.5625, 'eval_precision': 0.7029428904428905, 'eval_recall': 0.5625, 'eval_f1': 0.5507575757575758, 'eval_runtime': 1.8328, 'eval_samples_per_second': 34.918, 'eval_steps_per_second': 1.091, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.960433840751648, 'eval_accuracy': 0.96875, 'eval_precision': 0.9705882352941176, 'eval_recall': 0.96875, 'eval_f1': 0.9687194525904204, 'eval_runtime': 1.8237, 'eval_samples_per_second': 35.094, 'eval_steps_per_second': 1.097, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.801258385181427, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 2.2453, 'eval_samples_per_second': 28.504, 'eval_steps_per_second': 0.891, 'epoch': 3.0}
{'train_runtime': 92.3929, 'train_samples_per_second': 8.312, 'train_steps_per_second': 0.26, 'train_loss': 1.0827481746673584, 'epoch': 3.0}
>>>>>>>>>>>>> elapsed time: 2m


## Evaluación

In [ ]:
# Evaluación del modelo en el conjunto de prueba

# Preprocesamos el conjunto de prueba
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)

# Evaluamos el modelo en el conjunto de prueba
results = trainer.evaluate(test_preprocessed_dataset)

# Imprimimos las métricas de evaluación
print("Resultados de la evaluación en el conjunto de prueba:")
print(f"Exactitud: {results['eval_accuracy']:.4f}")
print(f"Precisión: {results['eval_precision']:.4f}")
print(f"Recall: {results['eval_recall']:.4f}")
print(f"F1-Score: {results['eval_f1']:.4f}")

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Resultados de la evaluación en el conjunto de prueba:
Exactitud: 0.9875
Precisión: 0.9881
Recall: 0.9875
F1-Score: 0.9875


In [24]:
# Guardamos el modelo y el tokenizer
save_directory = os.path.join(MODELS_LOCATION, 'disease_classification_spanish_nlp')
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modelo y tokenizer guardados en el directorio {save_directory}")

Modelo y tokenizer guardados en el directorio c:\Users\maria\Desktop\universidad\master\TFM\tfm\src\models\disease_classification_spanish_nlp


In [26]:
# Definamos algunas frases de ejemplo para el test que puedan confundir a nuestro modelo
sample_sentences = [
    {"Síntomas": "Me siento cansado todo el tiempo, como si no tuviera energía para nada.", "Enfermedad": "Anemia"},
    {"Síntomas": "Mi hijo no crece como debería y siempre se queja de estar débil.", "Enfermedad": "Talasemia"},
    {"Síntomas": "Tengo dolores en los huesos y me siento agotado incluso después de descansar.", "Enfermedad": "Talasemia"},
    {"Síntomas": "Tengo mucha sed todo el tiempo y no dejo de ir al baño.", "Enfermedad": "Diabetes"},
    {"Síntomas": "Tengo un dolor punzante y la pierna está hinchada y roja.", "Enfermedad": "Trombosis"},
    {"Síntomas": "He bajado de peso sin razón y me siento muy cansado.", "Enfermedad": "Diabetes"}
]

# Convertiamo le frasi di esempio in un DataFrame
sample_df = pd.DataFrame(sample_sentences)

# Tokenizziamo le frasi di esempio
sample_dataset = Dataset.from_pandas(sample_df)
sample_preprocessed_dataset = sample_dataset.map(preprocess_function, batched=True)

# Effettuiamo le predizioni
sample_predictions = trainer.predict(sample_preprocessed_dataset)
sample_preds = np.argmax(sample_predictions.predictions, axis=1)
sample_df["Enfermedad predicha"] = label_encoder.inverse_transform(sample_preds)

# Mostriamo i risultati
print("Resultados de las predicciones en las frases de ejemplo:")
sample_df

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Resultados de las predicciones en las frases de ejemplo:


,Síntomas,Enfermedad,Enfermedad predicha
0,"Me siento cansado todo el tiempo, como si no t...",Anemia,Anemia
1,Mi hijo no crece como debería y siempre se que...,Talasemia,Talasemia
2,Tengo dolores en los huesos y me siento agotad...,Talasemia,Talasemia
3,Tengo mucha sed todo el tiempo y no dejo de ir...,Diabetes,Diabetes
4,Tengo un dolor punzante y la pierna está hinch...,Trombosis,Talasemia
5,He bajado de peso sin razón y me siento muy ca...,Diabetes,Diabetes
